In [1]:
import torch.nn as nn
import torch

In [2]:
class ConsecutiveDifferenceLossAndBatch(nn.Module):
    def __init__(self, consecutive_size):
        super(ConsecutiveDifferenceLossAndBatch, self).__init__()
        self.consecutive_size = consecutive_size

    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        pred_dif = pred_reshape[:, 1:] - pred_reshape[:, :-1]
        target_dif = target_reshape[:, 1:] - target_reshape[:, :-1]
        
        pred_dif_batch = pred_reshape[1:, :] - pred_reshape[:-1, :]
        target_dif_batch = target_reshape[1:, :] - target_reshape[:-1, :]
        
        consec_loss = torch.mean((pred_dif - target_dif) ** 2)
        batch_loss = torch.mean((pred_dif_batch - target_dif_batch) ** 2)
        
        return consec_loss + batch_loss

In [3]:
class ConsecutiveSlopeWeightedMSELoss(nn.Module):
    def __init__(self, consecutive_size):
        super(ConsecutiveSlopeWeightedMSELoss, self).__init__()
        """this is just mse weighted by how different the slope before and after the point is"""
        self.consecutive_size = consecutive_size

    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        #pred_dif = pred_reshape[:, 1:] - pred_reshape[:, :-1]
        target_slope = target_reshape[:, 1:] - target_reshape[:, :-1]
        
        target_slope_dif = target_slope[:, 1:] - target_slope[:, :-1]
        
        
        errors = pred_reshape[:,1:-1] - target_reshape[:,1:-1]
        
        return torch.mean(target_slope_dif **2 * errors **2)
    

In [82]:
class ConsecutiveBorderCrossingAbs(nn.Module):
    def __init__(self, consecutive_size, temperature = 1.0):
        super(ConsecutiveBorderCrossingAbs, self).__init__()
        """just looks at where it crosses zero, also, do i even need tanh?"""
        self.consecutive_size = consecutive_size
        self.bce_loss = nn.BCELoss()
        self.temp = temperature

    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        target_sign_tensor = torch.sign(target_reshape)
        target_sign_diff = (target_sign_tensor[:,1:] - target_sign_tensor[:,:-1])/2 #-1,0,and 1 only
        target_sign_diff = (target_sign_diff + 1)/2 #bce ready
        
        pred_abs = torch.abs(pred_reshape)
        pred_sign_reshape = pred_reshape/pred_abs
        pred_sign_dif = pred_sign_reshape[:,1:] - pred_sign_reshape[:,:-1] #-2,0,2
        tan_pred = torch.tanh(self.temp * pred_sign_dif)
        bce_ready_pred = (tan_pred + 1)/2
        loss = self.bce_loss(bce_ready_pred,target_sign_diff)
        
        return loss
    
class ConsecutiveBorderCrossingAbs(nn.Module):
    def __init__(self, consecutive_size, temperature = 1.0):
        super(ConsecutiveBorderCrossingAbs, self).__init__()
        """just looks at where it crosses zero, also, do i even need tanh?"""
        self.consecutive_size = consecutive_size
        self.bce_loss = nn.BCELoss()
        self.temp = temperature

    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        target_sign_tensor = torch.sign(target_reshape)
        target_sign_diff = (target_sign_tensor[:,1:] - target_sign_tensor[:,:-1])/2 #-1,0,and 1 only
        target_sign_diff = (target_sign_diff + 1)/2 #bce ready
        
        pred_abs = torch.abs(pred_reshape)
        pred_sign_reshape = pred_reshape/pred_abs
        pred_sign_dif = pred_sign_reshape[:,1:] - pred_sign_reshape[:,:-1] #-2,0,2
        tan_pred = torch.tanh(self.temp * pred_sign_dif)
        bce_ready_pred = (tan_pred + 1)/2
        loss = self.bce_loss(bce_ready_pred,target_sign_diff)
        
        return loss

In [4]:
batch_size = 4
consecutive_size = 5

# Randomly generated data to simulate predictions and targets
prediction = torch.randn(batch_size, consecutive_size)
target = torch.randn(batch_size, consecutive_size)

# Instantiate the loss function
loss_fn = ConsecutiveSlopeWeightedMSELoss(consecutive_size)

# Calculate the loss
loss = loss_fn(prediction, target)
loss

tensor(20.7150)

In [5]:
prediction,target

(tensor([[ 0.8797, -0.3001,  1.5628, -0.2552, -2.6651],
         [ 2.0286,  0.2365, -0.1973,  1.5550, -0.2282],
         [-0.8787, -0.5304, -2.1451,  1.5823,  1.1772],
         [ 0.4714, -0.3693, -0.7140, -1.7759,  1.1688]]),
 tensor([[-0.8183,  0.3714, -2.1213, -0.6426, -0.8406],
         [-0.0229, -0.6064, -0.2235,  1.0801,  0.4553],
         [ 0.1327, -1.4317, -0.2237,  1.0804,  1.3824],
         [ 0.9487,  1.6239,  0.1116, -0.8401, -0.8859]]))

In [53]:

target_reshape = target.view(-1, consecutive_size)
print(target_reshape)
sign_tensor = torch.sign(target_reshape)
    
    # Compute the difference between adjacent sign elements
sign_diff = sign_tensor[:,1:] - sign_tensor[:,:-1]
sign_tensor,sign_diff/2

tensor([[-0.8183,  0.3714, -2.1213, -0.6426, -0.8406],
        [-0.0229, -0.6064, -0.2235,  1.0801,  0.4553],
        [ 0.1327, -1.4317, -0.2237,  1.0804,  1.3824],
        [ 0.9487,  1.6239,  0.1116, -0.8401, -0.8859]])


(tensor([[-1.,  1., -1., -1., -1.],
         [-1., -1., -1.,  1.,  1.],
         [ 1., -1., -1.,  1.,  1.],
         [ 1.,  1.,  1., -1., -1.]]),
 tensor([[ 1., -1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.],
         [-1.,  0.,  1.,  0.],
         [ 0.,  0., -1.,  0.]]))

In [52]:
tan_targets = torch.tanh(10*target_reshape)
tan_dif = tan_targets[:,1:] - tan_targets[:,:-1]
tan_dif/2

tensor([[ 9.9941e-01, -9.9941e-01,  2.6226e-06, -2.5630e-06],
        [-3.8765e-01,  1.1313e-02,  9.8868e-01, -1.1092e-04],
        [-9.3426e-01,  1.1274e-02,  9.8873e-01,  0.0000e+00],
        [ 0.0000e+00, -9.6855e-02, -9.0314e-01, -2.9802e-08]])

In [81]:
c = ConsecutiveBorderCrossingAbs(consecutive_size,temperature = 1.0)
c(target,target)

tensor(0.4400)

In [22]:
class ConsecutiveDifferenceHigherOrderLoss(nn.Module):
    def __init__(self, consecutive_size,order=1):
        super(ConsecutiveDifferenceHigherOrderLoss, self).__init__()
        self.consecutive_size = consecutive_size
        self.order = order
    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        result = torch.tensor([0.0])
        
        pred_a = pred_reshape[:, 1:]
        pred_b = pred_reshape[:, :-1]
        target_a = target_reshape[:, 1:]
        target_b = target_reshape[:, :-1]
        for i in range(self.order):
            print(i)
            pred_dif = pred_a - pred_b
            target_dif = target_a - target_b
            pred_a = pred_dif[:, 1:]
            pred_b = pred_dif[:, :-1]
            target_a = target_dif[:, 1:]
            target_b = target_dif[:, :-1]
            
            result +=  torch.mean((pred_dif - target_dif) ** 2)
        return result

In [33]:
class ConsecutiveDifferenceHigherOrderLossBatch(nn.Module):
    def __init__(self, consecutive_size,order=1):
        super(ConsecutiveDifferenceHigherOrderLossBatch, self).__init__()
        self.consecutive_size = consecutive_size
        self.order = order
    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        result = torch.tensor([0.0])
        
        pred_a = pred_reshape[ 1:, :]
        pred_b = pred_reshape[:-1,:]
        target_a = target_reshape[ 1:, :]
        target_b = target_reshape[:-1,:]
        for i in range(self.order):
            print(i)
            pred_dif = pred_a - pred_b
            target_dif = target_a - target_b
            pred_a = pred_dif[ 1:, :]
            pred_b = pred_dif[:-1,:]
            target_a = target_dif[ 1:, :]
            target_b = target_dif[:-1,:]
            
            result +=  torch.mean((pred_dif - target_dif) ** 2) /self.order
        return result

In [34]:
loss_fn = ConsecutiveDifferenceHigherOrderLossBatch(consecutive_size,order=3)

# Calculate the loss
loss = loss_fn(prediction, target)
loss

0
1
2


tensor([15.9454])

In [6]:
batch_size = 4
consecutive_size = 5

# Randomly generated data to simulate predictions and targets
prediction = torch.randn(batch_size, consecutive_size)
target = torch.randn(batch_size, consecutive_size)

# Instantiate the loss function
loss_fn = ConsecutiveSlopeWeightedMSELoss(consecutive_size)

# Calculate the loss
loss = loss_fn(prediction, target)
loss

NameError: name 'ConsecutiveSlopeWeightedMSELoss' is not defined